Pozostało nam poznać jeszcze kilka ważnych rzeczy, które są charakterystyczne dla Sparka a które sprawiają, że te narzędzie nadaje się idealnie do przetwarzania dużych zbiorów.

In [1]:
import pyspark
sc = pyspark.SparkContext('local[*]')

### Partycjonowanie

In [2]:
rdd1 = sc.parallelize([1,2,3,4,5]*100)
rdd1.getNumPartitions()

# można od razu przy tworzeniu:
# rdd1 = sc.parallelize([1,2,3,4,5]*100), numSlices=8)

12

In [3]:
rdd2=rdd1.repartition(8)
rdd2.getNumPartitions()

8

In [30]:
rdd2.glom().map(len).collect()

[61, 70, 72, 72, 62, 51, 52, 60]

In [31]:
rdd3=rdd1.map(lambda x:(x,x)).partitionBy(10).filter(lambda pair: pair[0] == 5)
print(rdd3.glom().map(len).collect())

[0, 0, 0, 0, 0, 100, 0, 0, 0, 0]


`partitionBy(k)` - dla RDD (key, value) rozrzuca RDD na k partycji. Wybór partycji wg: `key % k`

Inny sposób rozrzucania danych po work'erach:

In [32]:
rdd4 = rdd3.repartition(5) # losowo po 5 partycjach
print(rdd4.repartition(5).glom().map(len).collect())

[10, 10, 30, 30, 20]


Jest też inny sposób zmiany partycji jakie przetrzymują dane. Metoda nazywa się `coalesce`. Działa ona tylko przy zmniejszaniu ilości partycji. Jej sposób działania (w odróżnieniu do repartition) powoduje tylko przeniesienie danych tylko z partycji usuwanych. Dane w innych partycjach nie zmianiają miejsca. Jest to dość dobre z perspektywy wydajnościowej działa szybciej.

Dane:
```
Partition A: 1, 2, 3
Partition B: 4, 5, 6
Partition C: 7, 8, 9
Partition D: 10, 11, 12
```

`repartition(2):`
```
Partition E: 1, 3, 4, 6, 7, 9, 10, 12
Partition F: 2, 5, 8, 11
```

`coalesce(2):`

```
Partition G: 1, 2, 3, + (7, 8, 9)
Partition H: 4, 5, 6, + (10, 11, 12)
```

### Lazy Evaluation (Leniwa ewaluacja/wartościowanie)

In [33]:
%%time
print("Komenda %%time liczy czas wykonania danej komórki Jupytera")

Komenda %%time liczy czas wykonania danej komórki Jupytera
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 318 µs


Jak to jest z tą leniwą ewaluacją w Sparku? Spójrzmy na przykłady, które ukazują tą właściwość

In [34]:
%%time
array = [1,2,3,4,5]*100
# 500 elementów w tablicy

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 29.1 µs


In [35]:
%%time
rdd = sc.parallelize(array)
# czas utworzenia RDD

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 5.64 ms


In [36]:
print(rdd.toDebugString().decode())

(12) ParallelCollectionRDD[54] at parallelize at PythonRDD.scala:184 []


In [37]:
from math import sin, cos
def dlugaFunkcja(x):
    for i in range(100000):
        sin(x)**2 + cos(x)**2
    return x

In [38]:
%%time
dlugaFunkcja(80)

CPU times: user 80 ms, sys: 4 ms, total: 84 ms
Wall time: 81.7 ms


80

In [39]:
%%time
firstResult=rdd.map(lambda x: dlugaFunkcja(x))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 34.6 µs


In [40]:
print(firstResult.toDebugString().decode()) # dependence graph

(12) PythonRDD[55] at RDD at PythonRDD.scala:49 []
 |   ParallelCollectionRDD[54] at parallelize at PythonRDD.scala:184 []


In [41]:
%%time
firstResult.reduce(lambda x,y: x + y)

CPU times: user 8 ms, sys: 8 ms, total: 16 ms
Wall time: 3.23 s


1500

In [42]:
%%time
print(firstResult.count())

500
CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 3.32 s


### Cache'owanie wyników

In [49]:
%%time

# no to jeszcze raz:
firstResult=rdd.map(lambda x: dlugaFunkcja(x))

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 34.1 µs


In [50]:
print(firstResult.toDebugString().decode())

(12) PythonRDD[61] at RDD at PythonRDD.scala:49 []
 |   ParallelCollectionRDD[54] at parallelize at PythonRDD.scala:184 []


In [51]:
firstResult.cache()

PythonRDD[61] at RDD at PythonRDD.scala:49

In [52]:
print(firstResult.toDebugString().decode())

(12) PythonRDD[61] at RDD at PythonRDD.scala:49 [Memory Serialized 1x Replicated]
 |   ParallelCollectionRDD[54] at parallelize at PythonRDD.scala:184 [Memory Serialized 1x Replicated]


In [53]:
%%time
firstResult.reduce(lambda x,y: x + y)

CPU times: user 4 ms, sys: 8 ms, total: 12 ms
Wall time: 3.3 s


1500

In [54]:
%%time
firstResult.count()

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 76.7 ms


500

Pamiętajcie proszę, że:

firstResult.cache() == firstResult.persist(pyspark.StorageLevel.MEMORY_ONLY)

In [23]:
from pyspark import StorageLevel
StorageLevel.DISK_ONLY
StorageLevel.DISK_ONLY_2
StorageLevel.MEMORY_AND_DISK
StorageLevel.MEMORY_AND_DISK_2
StorageLevel.MEMORY_AND_DISK_SER
StorageLevel.MEMORY_AND_DISK_SER_2
StorageLevel.MEMORY_ONLY   #   <-- domyślny
StorageLevel.MEMORY_ONLY_2
StorageLevel.MEMORY_ONLY_SER
StorageLevel.OFF_HEAP

StorageLevel(True, True, True, False, 1)

Opis typów: [tutaj](https://spark.apache.org/docs/latest/rdd-programming-guide.html#rdd-persistence)